# Gorth09

In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from klefki.types.algebra.concrete import (
    EllipticCurveGroupSecp256k1 as ECG,
    EllipticCurveCyclicSubgroupSecp256k1 as CG,
    FiniteFieldSecp256k1 as F,
    FiniteFieldCyclicSecp256k1 as CF
)

from operator import add
G = CG.G

import random

N = 0xFFFFF
random_f = lambda: CF(random.randint(1, N) % CF.P)

q = random_f()
H = G @ q
    
Gs = [G @ random_f() for _ in range(0, 256)]

In [3]:
rand = lambda: random.randint(1, 100)

In [255]:
import numpy as np

Mat = np.matrix
Hada = lambda x, y: Mat(np.array(x) * np.array(y))

## 3 The Schwartz-Zippel Lemma



* Lemma 1 (Schwartz-Zippel). Let poly be a non-zero multivariate polynomial of degree $d$ over $\mathbb{Z}_p$ , then the probability of $poly(x_1 ; \cdots ; x_m ) = 0$ for randomly chosen $x_1 ; \cdots ; x_m) \leftarrow \mathbb{Z}_p$ is at most $d/p$.

The Schwartz-Zippel lemma is frequently used in polynomial identity testing. Given two multi-variate poly- nomials $poly_1$ and $poly_2$ we can test whether $poly_1 (x_1 ; \cdots ,x_m )-poly_2 (x_1,\cdots,x_ m ) = 0$ for random $x_1,\cdots ,x_m \leftarrow \mathbb{Z}_p$ . If the two polynomials are identical this will always be true, whereas if the two polyno- mials are different then there is only probability $max(d_1,d_2 )=p$ for the equality to hold.

## 4 Equations with Matrices and Vectors

We wish to commit to matrices and vectors of elements from $\mathbb{Z}_p$ and make SHVZK arguments for them satisfying equations commonly arising in linear algebra. We ﬁrst consider the following 6 types of equations over committed matrices $X_i ; Y_i ; Z \in Mat_{nxn} (\mathbb{Z}_p )$, committed row vectors $x_i,  y_i ,z \in \mathbb{Z}_p^n$ and committed elements $z\in \mathbb{Z}_p$ , with public $a^i \in \mathbb{Z}_p$ .

In [399]:
n = 5
m = 10
X = [[[random_f() for i in range(n)] for i in range(n)] for _ in range(m)]
Y = [[[random_f() for i in range(n)] for i in range(n)] for _ in range(m)]
a = [random_f() for _ in range(n*m)]
x = lambda e: [i[e] for i in X]
y = lambda e: [i[e] for i in Y]

In [400]:
from klefki.zkp.pedersen import com
from operator import add
from functools import reduce

$$
\mathbf{z}^\top = \sum_{i=1}^m a_iX_i\mathbf{y}_i^\top
$$

$$
z=\sum_{i=1}^m a_i \mathbf{x}_i \mathbf{y}_i^\top
$$

In [402]:
zs_T = reduce(add, [(Mat(_X) * _a) * Mat(_y).T
                    for (_X, _y, _a) in zip(X, y(0), a)])

In [482]:
z = lambda e: reduce(add, [(Mat(_x) * _a) * Mat(_y).T 
                 for (_x, _y, _a) in zip(x(e), y(e), a)]).item()

In [483]:
z(0) in zs_T

True

In [336]:
zs_T.T.tolist()[0]

[FiniteFieldCyclicSecp256k1::7650178624621212339,
 FiniteFieldCyclicSecp256k1::6874493472052062519,
 FiniteFieldCyclicSecp256k1::6909257848906660478,
 FiniteFieldCyclicSecp256k1::6703128170480367297,
 FiniteFieldCyclicSecp256k1::6147631864442045779]

In [331]:
reduce(add,
    [[reduce(add,
            [(Mat(_x) * _a) * Mat(_y).T 
                for (_x, _y, _a) in zip(x(e), y(e), a)]).item()
        for e in range(n)]])

[FiniteFieldCyclicSecp256k1::7650178624621212339,
 FiniteFieldCyclicSecp256k1::7195236569458980910,
 FiniteFieldCyclicSecp256k1::7378312022077696287,
 FiniteFieldCyclicSecp256k1::6253098340180404065,
 FiniteFieldCyclicSecp256k1::6175985997862321447]

$$
Z=\sum_{i=1}^m a_i X_iY_i \\
\mathbf{z}=\sum_{i=1}^m a_i \mathbf{x}_i Y_i \\
$$

In [245]:
Z = reduce(add, [Mat(_X) * _a * Mat(_Y)
                 for (_X, _Y, _a) in zip(X, Y, a)])

In [246]:
zs = reduce(add, [Mat(_x) * _a * Mat(_Y)
                  for (_x, _Y, _a) in zip(x, Y, a)])

In [247]:
zs in Z

True

$$
Z=\sum_{i=1}^m a_i X_i \circ Y_i \\
\mathbf{z}=\sum_{i=1}^m a_i \mathbf{x}_i \circ Y_i \\
$$

In [261]:
Z_h = reduce(add, [Hada(Mat(_X), Mat(_Y)) * _a
                 for (_X, _Y, _a) in zip(X, Y, a)])

In [262]:
zs_h = reduce(add, [Hada(Mat(_x), Mat(_Y)) * _a 
                  for (_x, _Y, _a) in zip(x, Y, a)])

In [263]:
zs_h in Z_h

True

### 4.1 Reducing Many Equations of the Form $z=\sum_{i=1}^m a_i \mathbf{x}_i \mathbf{y}_i^\top
$ to a Single Equation

Randomization can be used to reduce $Q$ equations of the form $z=\sum_{i=1}^{m_q} a_i \mathbf{x}_{qi} \mathbf{y}_{qi}^\top$ to one single equation of the form $z=\sum_{i=1}^m \mathbf{x}_i \mathbf{y}_i^\top$, where $m=\sum_{q=1}^Qmq$

The verifier selects $(r_1,\cdots,r_Q)\leftarrow Van_Q(\mathbb{Z}_p)$, 

In [477]:
q = n
r0 = random_f()
r = [r0 ** i for i in range(1, q+1)]

and only requires the prover to demonstrate:

$$
\sum_{q=1}^Q r_q z_q = \sum_{q=1}^Q\sum_{i=1}^{m_q}(r_q a_{qi}\mathbf{x}_{qi})\mathbf{y}^\top_{qi}
$$

This is a comparison of two degree $Q-1$ polynomials in the challenge consisting of a ﬁeld element.By the Schwartz-Zippel lemma, there is probability at most $\frac{Q-1}{p}$ for the test to pass unless indeed all the equations hold.

The probability of $\sum_{q=1}^Q r_q = 0$ is $\frac{Q-1}{p}$

By set $z=\sum_{q=1}^{Q}r_q z_q$, $x'=r_{q}a_{qi}\mathbf{x}_{qi}$, we get:

$$
z=\sum_{q=1}^Q\sum_{i=1}^{m_q} \mathbf{x}'_{qi} \mathbf{y}^\top_{qi}
$$

In [517]:
# z
z_ = reduce(add, [r[i] * z(i) for i in range(q)])

In [518]:
reduce(add, [reduce(add, [Mat(x(j)[i]) * r[j] * a[i] * Mat(y(j)[i]).T for i in range(m)])
 for j in range(q)]).tolist()[0][0]

FiniteFieldCyclicSecp256k1::354482359495557398002695090516611109927605351750

In [519]:
x_ = lambda q, i: Mat(x(q)[i]) * r[q] * a[i]

In [522]:
assert z_ == reduce(add, [reduce(add, [x_(i, j) * Mat(y(i)[j]).T 
       for j in range(m)])
       for i in range(q)]).item()